In [ ]:
import datetime
import collections

import tqdm
import tqdm.notebook
import pandas as pd
import numpy as np
from nile.api.v1 import (
    filters as nf,
    aggregators as na,
    extractors as ne,
    Record
)
from qb2.api.v1 import (filters as qf, typing)
from matplotlib import pyplot as plt
from IPython.display import display

from projects.common.nile import test_utils, dates
from projects.k2.support import project_config, default_values, common

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 200

In [ ]:
DT_START = '2020-11-01'
DT_END = '2021-07-15'

In [ ]:
PROJECT_PATH = '//home/taxi-delivery/analytics/dev/k2/support/{}'

K2_ORDERS_PATH = (
    '//home/market/production/mstat/analyst/regular/cubes_vertica/fact_new_order_dict_flattened'
)

MARKET_DATA_PATH = (
    '//home/market/production/mstat/dwh/calculation/fact_ocrm_communication_log_flattened/{}'
)
LAVKA_DATA_PATH = (
    '//home/taxi/production/services/support-eda/support_chatterbox_eats'
)
GO_DATA_PATH = (
    '//home/taxi-analytics/hickinbottom/support/dwh/summary/support_tickets'
)

In [ ]:
cluster = project_config.get_project_cluster(
    parallel_operations_limit=5,
    extra_requirements=[]
)

### K2 orders

In [ ]:
job = cluster.job()

In [ ]:
orders = job.table(K2_ORDERS_PATH)

orders = (
    orders
    .filter(
        qf.defined(
            'creation_datetime',
            'delivery_region_own_name',
            'multiorder_id',
            'id',
            'delivery_service_id',
            'delivered_flag',
        ),
        nf.custom(
            lambda x: DT_START < test_utils.to_string(x)[:10] <= DT_END,
            'creation_datetime'
        ),
        nf.custom(lambda x: x in {1005471, 1006308, 1006419, 1006422, 1006425, 1006428},
                  'delivery_service_id'),
    )
    .project(
        'creation_datetime',
        'delivered_flag',
        'delivery_region_own_name',
        'multiorder_id',
        order_id='id',
    )
    .put(PROJECT_PATH.format('orders_{}_{}'.format(DT_START, DT_END)))
)

multiorders = (
    orders
    .groupby('multiorder_id')
    .aggregate(
        creation_datetime=na.min('creation_datetime'),
        delivered_flg=na.min('delivered_flag'),
        delivery_region_own_name=na.any('delivery_region_own_name'),
        order_cnt=na.count(),
    )
    .put(PROJECT_PATH.format('multiorders_{}_{}'.format(DT_START, DT_END)))
)

In [ ]:
job.run()

In [ ]:
df_multiorders = test_utils.decode_dataframe(
    cluster
    .read(PROJECT_PATH.format('multiorders_{}_{}'.format(DT_START, DT_END)))
    .as_dataframe()
)

In [ ]:
df_multiorders['creation_dt'] = df_multiorders['creation_datetime'].apply(lambda x: x[:10])

In [ ]:
df_multiorders.head()

In [ ]:
df_multiorders.shape

### Deal with tickets

In [ ]:
job = cluster.job()

In [ ]:
orders = job.table(
    PROJECT_PATH.format('orders_{}_{}'.format(DT_START, DT_END))
)

go_data_proc = (
    job.table(GO_DATA_PATH)
    .filter(
        nf.custom(
            lambda x: 'taxi_kkd_ndd' in test_utils.to_string(x)
            if x else False,
            'line'
        ),
        nf.custom(
            lambda x: DT_START < test_utils.to_string(x) <= DT_END
            if x else False,
            'creation_dt'
        ),
    )
    .project(
        'ticket_id', 'autoreply_flg', 'category', 'chatterbox_ticket', 'city',
        'country', 'creation_dttm', 'csat', 'csat_flg', 'csat_value',
        'meta_category', 'order_id', 'reopens', 'sector', 'sla_full',
        'sla_reply', 'solved_dttm', 'source', 'status', 'tariff', 'theme',
        'ticket_channel_type', 'ticket_type', 'user_phone_id',
        'zendesk_ticket', 'line',
    )
     .put(PROJECT_PATH.format('test_tickets_go_{}_{}'.format(DT_START, DT_END)))
)

lavka_data_proc = (
    job.table(LAVKA_DATA_PATH)
    .filter(
        nf.or_(
            nf.equals('line', b'lavka_ndd_online'),
            nf.equals('line', b'lavka_ndd') 
        ),
        nf.custom(
            lambda x:  DT_START < str(datetime.datetime.fromtimestamp(x))[:10] <= DT_END
            if x else False,
            'created'
        ),
    )
    .project(
        'id', 'chat_type', 'external_id', 'first_answer', 'full_resolve',
        'history', 'line', 'meta_info', 'projects', 'reopen_count', 'startrack_ticket',
        'startrack_ticket_status', 'status', 'type', 'updated', 'zendesk_ticket',
        'zendesk_ticket_status',
        created_dttm=ne.custom(
            lambda x:  str(datetime.datetime.fromtimestamp(x))[:19]
            if x else None,
            'created'
        )
    )
    .put(PROJECT_PATH.format('test_tickets_lavka_{}_{}'.format(DT_START, DT_END)))
)

market_data_proc = (
    job.table(MARKET_DATA_PATH.format(dates.range_selector(DT_START, DT_END, '%Y-%m')))
    .filter(
        nf.custom(
            lambda x: DT_START < test_utils.to_string(x) <= DT_END
            if x else False,
            'ticket_registered_datetime'
        ),
        qf.defined('ticket_order_number'),
    )
    .join(orders, by_left='ticket_order_number', by_right='order_id', type='inner')
    .project(
        'ticket_reopened_datetime', 'ticket_start_processing_datetime',
        'datetime', 'ticket_time_in_queue', 'ticket_next_status_name', 'title',
        'ticket_taking_time', 'ocrm_team_title', 'measure_group', 'call_ended_at_datetime', 
        'call_start_status_datetime', 'ticket_csat_form_filled', 'resolution_name', 'status_name',
        'ticket_start_status_datetime', 'call_creation_datetime', 'start_status_datetime',
        'ticket_order_number', 'ticket_resolution_time', 'ticket_resolved_datetime',
        'ticket_service_id', 'ticket_registered_datetime', 'ticket_id', 'ticket_title',
        'call_ended_by', 'ticket_processing_time', 'ticket_time_in_status',
        'ticket_next_status_datetime', 'resolution_status', 'ticket_csat_score',
        'resolution_creation_time', 'call_end_call_status', 'resolved_counter',
        'ocrm_service_service_title', 'ticket_prev_status_datetime', 'ticket_prev_status_name',
        'ticket_last_resolved_datetime', 'ticket_status_name',
    )
     .put(PROJECT_PATH.format('test_tickets_market_{}_{}'.format(DT_START, DT_END)))
)

In [ ]:
job.run()

In [ ]:
df_go = test_utils.decode_dataframe(
    cluster.read(
         PROJECT_PATH.format('test_tickets_go_{}_{}'.format(DT_START, DT_END))
    ).as_dataframe()
)

df_lavka = test_utils.decode_dataframe(
    cluster.read(
         PROJECT_PATH.format('test_tickets_lavka_{}_{}'.format(DT_START, DT_END))
    ).as_dataframe()
)

df_market = test_utils.decode_dataframe(
    cluster.read(
         PROJECT_PATH.format('test_tickets_market_{}_{}'.format(DT_START, DT_END))
    ).as_dataframe()
)

+ Название сервиса, в который обратился юзер (Лавка, Маркет, Go + 1 и 2 линии маркета, РО/претензионка)
+ Канал обращения (телефон, почта, чат, тг и тд)
+ Реопены
+ тематика обращения
+ CSAT
+ Время первого обращения
+ sla первого ответа саппорта
+ sla закрытия
+ user_phone_pd_id
+ флаг рероутинга

In [ ]:
df_go.head()

In [ ]:
df_go['user_phone_pd_id'] = None

In [ ]:
go_type_map = {
    'writing': 'writing',
}

In [ ]:
df_go_proc = pd.DataFrame()

df_go_proc['request_service_name'] = ['go']*len(df_go)
df_go_proc['line'] = df_go['line'].apply(lambda x: '[go]_' + x)
df_go_proc['ocrm_service_service_title'] = 'undefined_for_service'

df_go_proc['request_channel'] = df_go['ticket_channel_type'].apply(
    lambda x: go_type_map.get(x)
)
df_go_proc['reopen_count'] = df_go['reopens'].fillna(0).apply(
    lambda x: int(x) if x and not np.isnan(x) else 0
)
df_go_proc['theme'] = df_go['theme'].apply(
    lambda x: x.replace('Unknown', '').replace('::', '. ').strip('. ') if x else 'NA'
).apply(lambda x: '[go]_NA' if not x else x)
df_go_proc['csat_value'] = df_go['csat_value']
df_go_proc['request_dttm'] = df_go['creation_dttm'].apply(
    lambda x: x[:19].replace('T', ' ')
)

df_go_proc['sla_first_answer_sec'] = df_go['sla_reply'].apply(
    lambda x: int(x*60) if x and not np.isnan(x) else None
)
df_go_proc['sla_resolution_sec'] = df_go['sla_full'].apply(
    lambda x: int(x*60) if x and not np.isnan(x) else None
)
df_go_proc['user_phone_pd_id'] = df_go['user_phone_pd_id']
# df_go_proc['rerouted_flg'] = pd.isnull(df_go['user_phone_id'])
df_go_proc['order_id'] = None

In [ ]:
df_go_proc.head()

In [ ]:
df_lavka.head()

In [ ]:
df_lavka.shape

In [ ]:
lavka_csat_value_map = {
    'amazing': 5,
    'good': 4,
    'normal': 3,
    'bad': 2,
    'horrible': 1,
}
lavka_type_map = {
    'chat': 'writing',
    'startrack': 'writing',
}

In [ ]:
df_lavka_proc = pd.DataFrame()

df_lavka_proc['request_service_name'] = ['lavka']*len(df_lavka)
df_lavka_proc['line'] = df_lavka['line'].apply(lambda x: '[lavka]_' + x)
df_lavka_proc['ocrm_service_service_title'] = 'undefined_for_service'
df_lavka_proc['request_channel'] = df_lavka['type'].apply(lambda x: lavka_type_map.get(x))
df_lavka_proc['reopen_count'] = df_lavka['reopen_count'].fillna(0).astype(int)
df_lavka_proc['theme'] = df_lavka['meta_info'].apply(
    lambda x: x.get('theme_name', '[lavka]_NA').replace('::', '. ')
)
df_lavka_proc['csat_value'] = df_lavka['meta_info'].apply(
    lambda x: lavka_csat_value_map.get(x.get('csat_value'))
)
df_lavka_proc['request_dttm'] = df_lavka['created_dttm']
df_lavka_proc['sla_first_answer_sec'] = df_lavka['first_answer'].apply(
    lambda x: int(x) if x and not np.isnan(x) else None
)
df_lavka_proc['sla_resolution_sec'] = df_lavka['full_resolve'].apply(
    lambda x: int(x) if x and not np.isnan(x) else None
)
df_lavka_proc['user_phone_pd_id'] = df_lavka['meta_info'].apply(
    lambda x: x.get('user_phone_pd_id')
)
# df_lavka_proc['rerouted_flg'] = ~(df_lavka['line']=='lavka_ndd_online')
df_lavka_proc['order_id'] = None

In [ ]:
df_lavka_proc.head()

In [ ]:
df_market.head()

In [ ]:
df_market.shape

In [ ]:
ocrm_team_title_map = {
    '1ая линия - чаты': '[market]_first_line',
    '1ая линия - Исходящая телефония': '[market]_first_line',
    '1ая линия поддержки': '[market]_first_line',
    '1ая линия (телефония)': '[market]_first_line',
    'Логистическая поддержка': '[market]_second_line',
    '1ая линия телефония': '[market]_first_line',
    'Рекламации Покупок': '[market]_reclamation',
    'Рекламации Беру': '[market]_reclamation',
}

ocrm_team_title_2_channel_map = {
    '1ая линия - чаты': 'writing',
    '1ая линия - Исходящая телефония': 'voice',
    '1ая линия поддержки': 'writing',
    '1ая линия (телефония)': 'voice',
    'Логистическая поддержка': 'writing',
    '1ая линия телефония': 'voice',
    'Рекламации Покупок': ' writing',
    'Рекламации Беру': 'writing',
}

In [ ]:
ticket_list = []
for key, df_g in tqdm.notebook.tqdm(
    df_market.sort_values('start_status_datetime').groupby('ticket_order_number'),
    total=len(df_market['ticket_order_number'].unique())
):
    
    ticket_dict = {'request_service_name': 'market'}
    
    ticket_dict['line'] = ocrm_team_title_map.get(
        df_g['ocrm_team_title'].iloc[-1], 'unknown'
    )
    ticket_dict['ocrm_service_service_title'] = (
        df_g['ocrm_service_service_title'].iloc[0]
    )
    
    ticket_dict['request_channel'] = (
        'voice' if not df_g['call_creation_datetime'].dropna().empty
        else 'writing'
    )
    ticket_dict['reopen_count'] = int(
        df_g[
            (df_g['ticket_status_name']=='reopened')
            & (df_g['ticket_prev_status_name']=='resolved')
        ].shape[0]
    )
    ticket_dict['theme'] = df_g['title'].iloc[0]
    ticket_dict['csat_value'] = None
    ticket_dict['request_dttm'] = df_g[
        df_g['ticket_registered_datetime']!='1970-01-01 00:00:00'
    ]['ticket_registered_datetime'].min()[:19]

    min_registered_dttm = ticket_dict['request_dttm']
    min_processing_dttm = df_g[
        df_g['ticket_start_processing_datetime']!='1970-01-01 00:00:00'
    ]['ticket_start_processing_datetime'].dropna().min()
    max_resolved_dttm = df_g[
        df_g['ticket_resolved_datetime']!='1970-01-01 00:00:00'
    ]['ticket_resolved_datetime'].dropna().max()

    ticket_dict['sla_first_answer_sec'] = None
    if not pd.isna(min_processing_dttm):
        ticket_dict['sla_first_answer_sec'] = (
            datetime.datetime.strptime(min_processing_dttm[:19], '%Y-%m-%d %H:%M:%S')
            - datetime.datetime.strptime(min_registered_dttm[:19], '%Y-%m-%d %H:%M:%S')
        ).total_seconds()
        
    ticket_dict['sla_resolution_sec'] = None
    if not pd.isna(max_resolved_dttm):
        ticket_dict['sla_resolution_sec'] = (
            datetime.datetime.strptime(max_resolved_dttm[:19], '%Y-%m-%d %H:%M:%S')
            - datetime.datetime.strptime(min_registered_dttm[:19], '%Y-%m-%d %H:%M:%S')
        ).total_seconds()

    ticket_dict['user_phone_pd_id'] = None
#     ticket_dict['rerouted_flg'] = None
    ticket_dict['order_id'] = df_g['ticket_order_number'].min()
    
    ticket_list.append(ticket_dict)

In [ ]:
df_market_proc = pd.DataFrame(ticket_list)

In [ ]:
df_market_proc.head()

In [ ]:
df_market_proc[df_market_proc['sla_first_answer_sec']>df_market_proc['sla_resolution_sec']]

In [ ]:
display(df_go_proc.shape)
display(df_lavka_proc.shape)
display(df_market_proc.shape)

In [ ]:
df_requests = pd.concat([df_go_proc, df_lavka_proc, df_market_proc], axis=0)
df_requests['request_dt'] = df_requests['request_dttm'].apply(lambda x: x[:10])

In [ ]:
df_requests.shape

In [ ]:
df_requests.head()

In [ ]:
theme_map_df = pd.read_excel('./inputs/themes_map.xlsx')

In [ ]:
theme_map_df.head()

In [ ]:
theme_map = dict(zip(theme_map_df['theme'], theme_map_df['theme_group']))

In [ ]:
df_requests['theme_group'] = df_requests['theme'].apply(
    lambda x: theme_map.get(x, 'Другое')
)

### Push to GP

In [ ]:
import business_models

In [ ]:
gp_path = 'analyst.k2_support_requests_v0'

if business_models.greenplum.check_table_exists(gp_path):
    business_models.greenplum.remove(gp_path)

for i in range(2):
    business_models.greenplum.write(
        dataframe=df_requests,
        table_name=gp_path,
        if_exists='replace',
    )

In [ ]:
gp_path = 'analyst.k2_multiorders_v0'

if business_models.greenplum.check_table_exists(gp_path):
    business_models.greenplum.remove(gp_path)

for i in range(2):
    business_models.greenplum.write(
        dataframe=df_multiorders,
        table_name=gp_path,
        if_exists='replace',
    )

In [ ]:
df_requests[df_requests['request_service_name']=='lavka']['request_dt'].unique().max()